In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

In [4]:
cur_folder_name = os.getcwd().split('/')[-1]
if cur_folder_name != "Virny":
    os.chdir("../..")

print('Current location: ', os.getcwd())

Current location:  /Users/denys_herasymuk/UCU/4course_2term/Bachelor_Thesis/Code/Virny


# Multiple Models Interface Usage

In [5]:
import os
import pandas as pd

from virny.datasets import ACSIncomeDataset
from virny.custom_classes.metrics_composer import MetricsComposer
from virny.custom_classes.metrics_interactive_visualizer import MetricsInteractiveVisualizer

In [6]:
data_loader = ACSIncomeDataset(state=['GA'], year=2018, with_nulls=False, subsample_size=15_000, subsample_seed=42)
sensitive_attributes_dct = {'SEX': '2', 'RAC1P': ['2', '3', '4', '5', '6', '7', '8', '9'], 'SEX&RAC1P': None}

In [7]:
ROOT_DIR = os.path.join('docs', 'examples')
subgroup_metrics_df = pd.read_csv(os.path.join(ROOT_DIR, 'income_subgroup_metrics.csv'), header=0)
subgroup_metrics_df['Model_Name'] = (subgroup_metrics_df['Model_Name'] + '__alpha=' +
                                     subgroup_metrics_df['Intervention_Param'].astype(str))

In [8]:
model_names = subgroup_metrics_df['Model_Name'].unique()
models_metrics_dct = dict()
for model_name in model_names:
    models_metrics_dct[model_name] = subgroup_metrics_df[subgroup_metrics_df['Model_Name'] == model_name]

metrics_composer = MetricsComposer(models_metrics_dct, sensitive_attributes_dct)
models_composed_metrics_df = metrics_composer.compose_metrics()
models_composed_metrics_df.head()

,Metric,SEX,RAC1P,SEX&RAC1P,Model_Name
0,Accuracy_Parity,0.047756,0.074977,0.065217,LGBMClassifier__alpha=0.7
1,Aleatoric_Uncertainty_Parity,-0.039005,-0.011947,-0.009222,LGBMClassifier__alpha=0.7
2,Aleatoric_Uncertainty_Ratio,0.935159,0.979638,0.984220,LGBMClassifier__alpha=0.7
3,Equalized_Odds_FNR,0.030793,-0.110745,-0.052498,LGBMClassifier__alpha=0.7
4,Equalized_Odds_FPR,-0.021317,0.000952,-0.007008,LGBMClassifier__alpha=0.7


In [10]:
models_metrics_dct = dict()
for model_name in subgroup_metrics_df['Model_Name'].unique():
    models_metrics_dct[model_name] = subgroup_metrics_df[subgroup_metrics_df['Model_Name'] == model_name]

In [11]:
models_metrics_dct.keys()

dict_keys(['LGBMClassifier__alpha=0.7', 'LGBMClassifier__alpha=0.0', 'LGBMClassifier__alpha=0.4', 'LogisticRegression__alpha=0.0', 'LogisticRegression__alpha=0.7', 'LogisticRegression__alpha=0.4', 'MLPClassifier__alpha=0.0', 'MLPClassifier__alpha=0.7', 'MLPClassifier__alpha=0.4', 'RandomForestClassifier__alpha=0.4', 'RandomForestClassifier__alpha=0.7', 'RandomForestClassifier__alpha=0.0'])

## Metrics Visualization and Reporting

In [58]:
visualizer = MetricsInteractiveVisualizer(data_loader.X_data, data_loader.y_data,
                                          models_metrics_dct, models_composed_metrics_df,
                                          sensitive_attributes_dct=sensitive_attributes_dct)

In [59]:
visualizer.start_web_app()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
Keyboard interruption in main thread... closing server.


In [12]:
visualizer.stop_web_app()

Closing server running on port: 7860
